In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

Wed Jan 30 2019 

CPython 3.6.8
IPython 7.2.0

numpy 1.15.4
scipy 1.2.0
sklearn 0.20.2
pandas 0.24.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.20.5-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 16
interpreter: 64bit


In [5]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nolds
import data
import mne
import matplotlib.pyplot as plt

from data.data_files import CHANNEL_NAMES, DataKind, files_builder

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
DATA_ROOT = os.path.abspath(os.path.join(PROJ_ROOT, 'data'))
PROCESSED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'processed'))
RAW_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'raw'))
LABELED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'labeled'))
DURATIONS_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'durations'))
REC_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'recplots'))
DIRECT_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'direct'))
GAF_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'gaf'))
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
CHANNEL_NAMES = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
                 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
META_COLUMN_NAMES = ['freq', 'RESP_4W', 'RESP_FIN', 'REMISE_FIN', 'AGE', 'SEX', 'M_1',
       'M_4', 'M_F', 'délka léčby', 'lék 1', 'lék 2', 'lék 3', 'lék 4']
META_FILE_NAME = 'DEP-POOL_Final_144.xlsx'
# meta_df = pd.read_excel(os.path.join(RAW_ROOT, META_FILE_NAME), index_col='ID', names=META_COLUMN_NAMES)

raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '1a.fif'))
t = pd.DataFrame(raw_fif.get_data())
data = pd.DataFrame(np.transpose(t.values), columns=CHANNEL_NAMES)
data = np.transpose(data.values)

Opening raw data file /home/kovar/thesis_project/data/processed/1a.fif...
This filename (/home/kovar/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 19104 =      0.000 ...    76.416 secs
Ready.


<ipython-input-5-d3665bdb3882>:31: RuntimeWarning: This filename (/home/kovar/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '1a.fif'))


In [6]:
import logging
import os.path
import time
from collections import OrderedDict
import sys

import numpy as np
import torch.nn.functional as F
from torch import optim

from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import LossMonitor, MisclassMonitor, RuntimeMonitor
from braindecode.experiments.stopcriteria import MaxEpochs, NoDecrease, Or
from braindecode.datautil.iterators import BalancedBatchSizeIterator
from braindecode.torch_ext.constraints import MaxNormDefaultConstraint
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.models.deep4 import Deep4Net

# Prepare data

In [7]:
# Welford's algorithm for computing running mean and variance
def update(existingAggregate, newValues):
    (count, mean, M2) = existingAggregate
    for newValue in newValues: 
        count += 1
        delta = newValue - mean
        mean += delta / count
        delta2 = newValue - mean
        M2 += delta * delta2
        existingAggregate = (count, mean, M2)

    return (count, mean, M2)

def finalize(existingAggregate):
    (count, mean, M2) = existingAggregate
    (mean, variance) = (mean, M2/count) 
    if count < 2:
        return float('nan')
    else:
        return (mean, np.sqrt(variance))

In [4]:
from braindecode.datautil.signalproc import bandpass_cnt
low_cut_hz = 4 # Suggested 0 or 4
high_cut_hz = 38
def compute_sep(file, path, ww=256, maxl=np.infty, rescale=True, filter=True):
    minl = ww
    start = 0
    chunk_num = 0
    N = len(file.df['FP1'].values)
    while start+ww <= min(maxl, N):
        ret = np.zeros((len(CHANNEL_NAMES), ww))
        for i, channel in enumerate(CHANNEL_NAMES):
            data = file.df[channel].values
            r = data[start:start+ww]
            if 2*len(data) < minl + ww or r.shape[0]*r.shape[0] != ww*ww:
                print('Skipping, file ', file.name, ', time series too short: ', len(data))
                print('Or returned wrong shape: ', r.shape, start+ww)
                continue
            if rescale:
                r *= 1e6
            if filter:
                r = bandpass_cnt(r, low_cut_hz, high_cut_hz, 250, filt_order=3, axis=0)
            counts[i], means[i], M2s[i] = update((counts[i], means[i], M2s[i]), r)
            ret[i, :] = r
        np.save(
            os.path.join(path, ''.join((str(file.id), file.trial, '-', str(chunk_num), '.npy'))),
            file.df.values.T[:, start:start+ww], fix_imports=False)
        start += ww
        chunk_num += 1
    return r

In [87]:
counts, means, M2s = np.zeros(19), np.zeros(19), np.zeros(19)
final_means, final_stds = np.zeros(19), np.zeros(19)

import logging
mne.set_log_level(logging.ERROR)
for i, file in enumerate(files_builder(DataKind('processed'))):
    compute_sep(file, DIRECT_ROOT)
    print('Processed: ', i)
    
print('Finalizing...')
for i in np.arange(len(CHANNEL_NAMES)):
    (final_means[i], final_stds[i]) = finalize((counts[i], means[i], M2s[i]))

Processed:  0
Processed:  1
Processed:  2
Processed:  3
Processed:  4
Processed:  5
Processed:  6
Processed:  7
Processed:  8
Processed:  9
Processed:  10
Processed:  11
Processed:  12
Processed:  13
Processed:  14
Processed:  15
Processed:  16
Processed:  17
Processed:  18
Processed:  19
Processed:  20
Processed:  21
Processed:  22
Processed:  23
Processed:  24
Processed:  25
Processed:  26
Processed:  27
Processed:  28
Processed:  29
Processed:  30
Processed:  31
Processed:  32
Processed:  33
Processed:  34
Processed:  35
Processed:  36
Processed:  37
Processed:  38
Processed:  39
Processed:  40
Processed:  41
Processed:  42
Processed:  43
Processed:  44
Processed:  45
Processed:  46
Processed:  47
Processed:  48
Processed:  49
Processed:  50
Processed:  51
Processed:  52
Processed:  53
Processed:  54
Processed:  55
Processed:  56
Processed:  57
Processed:  58
Processed:  59
Processed:  60
Processed:  61
Processed:  62
Processed:  63
Processed:  64
Processed:  65
Processed:  66
Proce

In [88]:
print('Saving...')
for fn in os.listdir(DIRECT_ROOT):
    filepath = os.path.join(DIRECT_ROOT,fn)
    r = np.load(filepath)
    assert r.shape[0] == len(CHANNEL_NAMES)
    for i in np.arange(r.shape[0]):
        r[i, :] = (r[i, :]-final_means[i])/final_stds[i]
    np.save(filepath, r, fix_imports=False)

Saving...


In [8]:
def remove_middle(filenames, labels):
    ta = zip(filenames, labels)
    ta = np.array([(t, l) for t, l in ta], dtype=[('fname', 'S100'), ('label', 'int64')])
    ta = ta[:][(ta['label'] == -1) | (ta['label'] == 1)]
    ta['label'][ta['label'] == -1] = 0
    return ta['fname'].astype(str, copy=False), ta['label']

In [9]:
from data.data_files import CHANNEL_NAMES, DataKind, files_builder
from lib.braindecode.braindecode.datautil.splitters import split_into_two_sets
from sklearn.model_selection import GridSearchCV, train_test_split

fb = files_builder(DataKind('recplot'), subfolder=('vectors',))
seed = 123
fns = [fn[1] for fn in fb.file_names(include_path=True, subfolder=(), recursive=True)]
filenames, labels = remove_middle(fns, fb.get_labels(fns))
unique, counts = np.unique(labels, return_counts=True)
print('Overall distribution: ', dict(zip(unique, counts)))
training_filenames, validation_filenames, training_labels, validation_labels = \
    train_test_split(filenames, labels, test_size=0.3, random_state=seed)
test_filenames, validation_filenames, test_labels, validation_labels = \
    train_test_split(validation_filenames, validation_labels, test_size=0.5, random_state=seed)
assert len(training_filenames) == len(training_labels)
assert len(validation_filenames) == len(validation_labels)
unique, counts = np.unique(training_labels, return_counts=True)
print('Training distribution: ', dict(zip(unique, counts)))
unique, counts = np.unique(test_labels, return_counts=True)
print('Testing distribution: ', dict(zip(unique, counts)))
unique, counts = np.unique(validation_labels, return_counts=True)
print('Validation distribution: ', dict(zip(unique, counts)))

Overall distribution:  {0: 6801, 1: 6006}
Training distribution:  {0: 4718, 1: 4246}
Testing distribution:  {0: 1027, 1: 894}
Validation distribution:  {0: 1056, 1: 866}


In [10]:
train_set = np.array([np.load(os.path.join(REC_ROOT, fn)).squeeze().astype('float32') for fn in training_filenames])
test_set = np.array([np.load(os.path.join(REC_ROOT, fn)).squeeze().astype('float32') for fn in test_filenames])
valid_set = np.array([np.load(os.path.join(REC_ROOT, fn)).squeeze().astype('float32') for fn in validation_filenames])

In [11]:
from braindecode.datautil.signal_target import SignalAndTarget

train_set = SignalAndTarget(train_set, y=training_labels)
valid_set = SignalAndTarget(valid_set, y=validation_labels)
test_set = SignalAndTarget(test_set, y=test_labels)

# Define model, optimization, etc.

In [12]:
n_chans = 256
n_classes = 2
cuda = False
input_time_length = 256
n_epochs = 35
batch_size = 32

In [13]:
from braindecode.torch_ext.functions import safe_log, square
# log = logging.getLogger(__name__)
model = ShallowFBCSPNet(
    n_chans, n_classes, input_time_length=input_time_length,
    n_filters_time=40,
    filter_time_length=25,
    n_filters_spat=40,
    pool_time_length=75,
    pool_time_stride=15,
    final_conv_length='auto',
    conv_nonlin=square,
    pool_mode='mean',
    pool_nonlin=safe_log,
    split_first_layer=True,
    batch_norm=True,
    batch_norm_alpha=0.1,
    drop_prob=0.5,
).create_network()
# model = Deep4Net(n_chans, n_classes, input_time_length=input_time_length, final_conv_length=20).create_network()
if cuda:
    model.cuda()
print("Model: \n{:s}".format(str(model)), '\nNumber of params: ', sum(p.numel() for p in model.parameters()))

Model: 
Sequential(
  (dimshuffle): Expression(expression=_transpose_time_to_spat)
  (conv_time): Conv2d(1, 40, kernel_size=(25, 1), stride=(1, 1))
  (conv_spat): Conv2d(40, 40, kernel_size=(1, 256), stride=(1, 1), bias=False)
  (bnorm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_nonlin): Expression(expression=square)
  (pool): AvgPool2d(kernel_size=(75, 1), stride=(15, 1), padding=0)
  (pool_nonlin): Expression(expression=safe_log)
  (drop): Dropout(p=0.5)
  (conv_classifier): Conv2d(40, 2, kernel_size=(11, 1), stride=(1, 1))
  (softmax): LogSoftmax()
  (squeeze): Expression(expression=_squeeze_final_output)
) 
Number of params:  411602


In [14]:
from braindecode.torch_ext.optimizers import AdamW
from braindecode.torch_ext.schedulers import ScheduledOptimizer, CosineAnnealing
from braindecode.datautil.iterators import get_balanced_batches
from numpy.random import RandomState
rng = RandomState((2018,8,7))
# optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
# Need to determine number of batch passes per epoch for cosine annealing
n_updates_per_epoch = len(list(get_balanced_batches(len(train_set.X), rng, shuffle=True,
                                            batch_size=batch_size)))
scheduler = CosineAnnealing(n_epochs * n_updates_per_epoch)
# schedule_weight_decay must be True for AdamW
optimizer = ScheduledOptimizer(scheduler, optimizer, schedule_weight_decay=True)

# Manual trial-wise training

In [15]:
from braindecode.torch_ext.util import np_to_var, var_to_np
import torch.nn.functional as F

for i_epoch in range(n_epochs):
    i_trials_in_batch = get_balanced_batches(len(train_set.X), rng, shuffle=True,
                                            batch_size=batch_size)
    # Set model to training mode
    model.train()
    for i_trials in i_trials_in_batch:
        # Have to add empty fourth dimension to X
        batch_X = train_set.X[i_trials][:,:,:,None]
        batch_y = train_set.y[i_trials]
        net_in = np_to_var(batch_X)
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target = net_target.cuda()
        # Remove gradients of last backward pass from all parameters
        optimizer.zero_grad()
        # Compute outputs of the network
        outputs = model(net_in)
        # Compute the loss
        loss = F.nll_loss(outputs, net_target)
        # Do the backpropagation
        loss.backward()
        # Update parameters with the optimizer
        optimizer.step()

    # Print some statistics each epoch
    model.eval()
    print("Epoch {:d}".format(i_epoch))
    for setname, dataset in (('Train', train_set), ('Valid', valid_set)):
        # Here, we will use the entire dataset at once, which is still possible
        # for such smaller datasets. Otherwise we would have to use batches.
        net_in = np_to_var(dataset.X[:,:,:,None])
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(dataset.y)
        if cuda:
            net_target = net_target.cuda()
        outputs = model(net_in)
        loss = F.nll_loss(outputs, net_target)
        print("{:6s} Loss: {:.5f}".format(
            setname, float(var_to_np(loss))))
        predicted_labels = np.argmax(var_to_np(outputs), axis=1)
        accuracy = np.mean(dataset.y  == predicted_labels)
        print("{:6s} Accuracy: {:.1f}%".format(
            setname, accuracy * 100))

Epoch 0


RuntimeError: $ Torch: not enough memory: you tried to allocate 79GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1544199946412/work/aten/src/TH/THGeneral.cpp:201

In [54]:
import torch as th

model.eval()
# Collect all predictions and losses
all_preds = []
all_losses = []
batch_sizes = []
for batch_X, batch_y in iterator.get_batches(test_set, shuffle=False):
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    net_target = np_to_var(batch_y)
    if cuda:
        net_target = net_target.cuda()
    outputs = model(net_in)
    all_preds.append(var_to_np(outputs))
    loss = F.nll_loss(outputs, net_target)
    loss = float(var_to_np(loss))
    all_losses.append(loss)
    batch_sizes.append(len(batch_X))
# Compute mean per-input loss
loss = np.mean(np.array(all_losses) * np.array(batch_sizes) /
               np.mean(batch_sizes))
print("Test Loss: {:.5f}".format(loss))
# Assign the predictions to the trials
preds_per_trial = compute_preds_per_trial_from_crops(all_preds,
                                                  input_time_length,
                                                  test_set.X)
# preds per trial are now trials x classes x timesteps/predictions
# Now mean across timesteps for each trial to get per-trial predictions
meaned_preds_per_trial = np.array([np.mean(p, axis=1) for p in preds_per_trial])
predicted_labels = np.argmax(meaned_preds_per_trial, axis=1)
accuracy = np.mean(predicted_labels == test_set.y)
print("Test Accuracy: {:.1f}%".format(accuracy * 100))

ValueError: Expected 2 or more dimensions (got 1)

# Automatic trial-wise training

In [98]:
batch_size = 32
max_epochs = 35
max_increase_epochs = 4

optimizer = optim.Adam(model.parameters())

iterator = BalancedBatchSizeIterator(batch_size=batch_size)

stop_criterion = Or([MaxEpochs(max_epochs),
                     NoDecrease('valid_misclass', max_increase_epochs)])

monitors = [LossMonitor(), MisclassMonitor(), RuntimeMonitor()]

model_constraint = MaxNormDefaultConstraint()

exp = Experiment(model, train_set, valid_set, test_set, iterator=iterator,
                 loss_function=F.nll_loss, optimizer=optimizer,
                 model_constraint=model_constraint,
                 monitors=monitors,
                 stop_criterion=stop_criterion,
                 remember_best_column='valid_misclass',
                 run_after_early_stop=True, cuda=cuda)
exp.run()

In [99]:
print("Last 10 epochs")
print("\n" + str(exp.epochs_df.iloc[-10:]))

Last 10 epochs

    train_loss  valid_loss  test_loss  train_misclass  valid_misclass  \
45    0.313250    0.336083   0.372132        0.114367        0.127992   
46    0.295478    0.312901   0.361784        0.103252        0.119147   
47    0.342636    0.348469   0.419809        0.148815        0.146722   
48    0.309392    0.315402   0.381417        0.127503        0.126431   
49    0.312134    0.324736   0.381960        0.122818        0.129553   
50    0.318551    0.328828   0.376518        0.123186        0.134755   
51    0.291159    0.306112   0.357053        0.094984        0.106139   
52    0.295919    0.306773   0.359336        0.108580        0.119667   
53    0.298605    0.306146   0.363143        0.112071        0.113944   
54    0.291833    0.295663   0.357623        0.101415        0.106139   

    test_misclass    runtime  
45       0.161374  17.108581  
46       0.152004  17.179595  
47       0.187402  17.236296  
48       0.176471  17.242505  
49       0.161374  17.051

# Manual cropped training

In [70]:
in_chans = 19
n_classes = 2
cuda = False
input_time_length = 256
n_epochs = 50
max_epochs = n_epochs
max_increase_epochs = 4
batch_size = 32

In [71]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length,
                        final_conv_length='auto').create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()

In [72]:
from braindecode.torch_ext.util import np_to_var
# determine output size
# test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
test_input = np_to_var(train_set.X[:1, :, :, None])
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]

print("{:d} predictions per input/trial".format(n_preds_per_input))

from braindecode.datautil.iterators import CropsFromTrialsIterator
iterator = CropsFromTrialsIterator(batch_size=batch_size,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

from braindecode.torch_ext.optimizers import AdamW
from braindecode.torch_ext.schedulers import ScheduledOptimizer, CosineAnnealing
from braindecode.datautil.iterators import get_balanced_batches
from numpy.random import RandomState
rng = RandomState((2018,8,7))
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
# Need to determine number of batch passes per epoch for cosine annealing
n_updates_per_epoch = len([None for b in iterator.get_batches(train_set, True)])
scheduler = CosineAnnealing(n_epochs * n_updates_per_epoch)
# schedule_weight_decay must be True for AdamW
optimizer = ScheduledOptimizer(scheduler, optimizer, schedule_weight_decay=True)

8 predictions per input/trial


In [75]:
from braindecode.torch_ext.util import np_to_var, var_to_np
import torch.nn.functional as F
from numpy.random import RandomState
import torch as th
from braindecode.experiments.monitors import compute_preds_per_trial_from_crops
rng = RandomState((2017,6,30))
for i_epoch in range(n_epochs):
    # Set model to training mode
    model.train()
    for batch_X, batch_y in iterator.get_batches(train_set, shuffle=True):
        net_in = np_to_var(batch_X)
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target = net_target.cuda()
        # Remove gradients of last backward pass from all parameters
        optimizer.zero_grad()
        outputs = model(net_in)
        # Mean predictions across trial
        # Note that this will give identical gradients to computing
        # a per-prediction loss (at least for the combination of log softmax activation
        # and negative log likelihood loss which we are using here)
        outputs = th.mean(outputs, dim=2, keepdim=False)
        loss = F.nll_loss(outputs, net_target)
        loss.backward()
        optimizer.step()

    # Print some statistics each epoch
    model.eval()
    print("Epoch {:d}".format(i_epoch))
    for setname, dataset in (('Train', train_set),('Valid', valid_set)):
        # Collect all predictions and losses
        all_preds = []
        all_losses = []
        batch_sizes = []
        for batch_X, batch_y in iterator.get_batches(dataset, shuffle=False):
            net_in = np_to_var(batch_X)
            if cuda:
                net_in = net_in.cuda()
            net_target = np_to_var(batch_y)
            if cuda:
                net_target = net_target.cuda()
            outputs = model(net_in)
            all_preds.append(var_to_np(outputs))
            outputs = th.mean(outputs, dim=2, keepdim=False)
            loss = F.nll_loss(outputs, net_target)
            loss = float(var_to_np(loss))
            all_losses.append(loss)
            batch_sizes.append(len(batch_X))
        # Compute mean per-input loss
        loss = np.mean(np.array(all_losses) * np.array(batch_sizes) /
                       np.mean(batch_sizes))
        print("{:6s} Loss: {:.5f}".format(setname, loss))
        # Assign the predictions to the trials
        preds_per_trial = compute_preds_per_trial_from_crops(all_preds,
                                                          input_time_length,
                                                          dataset.X)
        # preds per trial are now trials x classes x timesteps/predictions
        # Now mean across timesteps for each trial to get per-trial predictions
        meaned_preds_per_trial = np.array([np.mean(p, axis=1) for p in preds_per_trial])
        predicted_labels = np.argmax(meaned_preds_per_trial, axis=1)
        accuracy = np.mean(predicted_labels == dataset.y)
        print("{:6s} Accuracy: {:.1f}%".format(
            setname, accuracy * 100))

Epoch 0
Train  Loss: 0.58090
Train  Accuracy: 69.3%
Valid  Loss: 0.66640
Valid  Accuracy: 61.9%
Epoch 1
Train  Loss: 0.50311
Train  Accuracy: 76.6%
Valid  Loss: 0.60199
Valid  Accuracy: 68.5%
Epoch 2
Train  Loss: 0.48061
Train  Accuracy: 77.1%
Valid  Loss: 0.60609
Valid  Accuracy: 68.3%
Epoch 3
Train  Loss: 0.42770
Train  Accuracy: 81.1%
Valid  Loss: 0.55219
Valid  Accuracy: 71.6%
Epoch 4
Train  Loss: 0.38405
Train  Accuracy: 83.8%
Valid  Loss: 0.52567
Valid  Accuracy: 76.1%
Epoch 5
Train  Loss: 0.34461
Train  Accuracy: 86.3%
Valid  Loss: 0.50529
Valid  Accuracy: 73.6%
Epoch 6
Train  Loss: 0.32876
Train  Accuracy: 86.8%
Valid  Loss: 0.47941
Valid  Accuracy: 77.5%
Epoch 7
Train  Loss: 0.36207
Train  Accuracy: 83.1%
Valid  Loss: 0.54554
Valid  Accuracy: 73.7%
Epoch 8
Train  Loss: 0.30392
Train  Accuracy: 87.5%
Valid  Loss: 0.47964
Valid  Accuracy: 77.3%
Epoch 9
Train  Loss: 0.25175
Train  Accuracy: 91.2%
Valid  Loss: 0.44010
Valid  Accuracy: 78.3%
Epoch 10
Train  Loss: 0.26492
Train  Acc

In [76]:
model.eval()
# Collect all predictions and losses
all_preds = []
all_losses = []
batch_sizes = []
for batch_X, batch_y in iterator.get_batches(test_set, shuffle=False):
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    net_target = np_to_var(batch_y)
    if cuda:
        net_target = net_target.cuda()
    outputs = model(net_in)
    all_preds.append(var_to_np(outputs))
    outputs = th.mean(outputs, dim=2, keepdim=False)
    loss = F.nll_loss(outputs, net_target)
    loss = float(var_to_np(loss))
    all_losses.append(loss)
    batch_sizes.append(len(batch_X))
# Compute mean per-input loss
loss = np.mean(np.array(all_losses) * np.array(batch_sizes) /
               np.mean(batch_sizes))
print("Test Loss: {:.5f}".format(loss))
# Assign the predictions to the trials
preds_per_trial = compute_preds_per_trial_from_crops(all_preds,
                                                  input_time_length,
                                                  test_set.X)
# preds per trial are now trials x classes x timesteps/predictions
# Now mean across timesteps for each trial to get per-trial predictions
meaned_preds_per_trial = np.array([np.mean(p, axis=1) for p in preds_per_trial])
predicted_labels = np.argmax(meaned_preds_per_trial, axis=1)
accuracy = np.mean(predicted_labels == test_set.y)
print("Test Accuracy: {:.1f}%".format(accuracy * 100))

Test Loss: 0.40381
Test Accuracy: 84.4%


# Automatic cropped training

In [73]:
from braindecode.experiments.monitors import LossMonitor, MisclassMonitor, RuntimeMonitor, CroppedTrialMisclassMonitor
from braindecode.datautil.iterators import CropsFromTrialsIterator
from braindecode.torch_ext.constraints import MaxNormDefaultConstraint
from braindecode.experiments.stopcriteria import MaxEpochs, NoDecrease, Or

optimizer = optim.Adam(model.parameters())
iterator = CropsFromTrialsIterator(batch_size=batch_size,
                                   input_time_length=input_time_length,
                                   n_preds_per_input=n_preds_per_input)
stop_criterion = Or([MaxEpochs(max_epochs),
                     NoDecrease('valid_misclass', max_increase_epochs)])
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(
                input_time_length=input_time_length), RuntimeMonitor()]

model_constraint = MaxNormDefaultConstraint()

loss_function = lambda preds, targets: F.nll_loss(
    th.mean(preds, dim=2, keepdim=False), targets)

exp = Experiment(model, train_set, valid_set, test_set, iterator=iterator,
                 loss_function=loss_function, optimizer=optimizer,
                 model_constraint=model_constraint,
                 monitors=monitors,
                 stop_criterion=stop_criterion,
                 remember_best_column='valid_misclass',
                 run_after_early_stop=True, cuda=cuda)
exp.run()

In [74]:
print("Last 20 epochs")
print("\n" + str(exp.epochs_df.iloc[-20:]))

Last 20 epochs

    train_loss  valid_loss  test_loss  train_sample_misclass  \
10    0.347806    0.437950   0.421939               0.140590   
11    0.335104    0.433781   0.419113               0.131791   
12    0.330339    0.434856   0.408854               0.130982   
13    0.355878    0.456078   0.442913               0.156250   
14    0.332419    0.445152   0.412524               0.130913   
15    0.315801    0.426716   0.399017               0.115615   
16    0.313600    0.428834   0.402357               0.120203   
17    0.414201    0.520785   0.523055               0.188351   
18    0.311591    0.411437   0.396985               0.117693   
19    0.329220    0.411437   0.396985               0.129915   
20    0.315685    0.371553   0.382071               0.115860   
21    0.328018    0.374032   0.392980               0.128422   
22    0.319208    0.359238   0.390692               0.122129   
23    0.318259    0.360736   0.386656               0.121969   
24    0.332172    0.3737